In [2]:
!pip install plotly
!pip install pandas

# Importing libraries
import plotly.offline as go_offline
import plotly.graph_objects as go
import pandas as pd

url = 'https://docs.google.com/spreadsheets/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/export?format=csv&id'
data = pd.read_csv(url)

# Cleaning data
data.drop(['First confirmed date in country (Est.)'], axis=1, inplace=True)

data.fillna("", inplace=True)
data.head()

# Combine region and country (since there are some empty spaces in the Province/State column)
def combine_region(row):
    province = row["Province/State"]
    country = row["Country/Region"]
    
    if province == country:
        return province
    
    if province == '':
        return country
    
    if province != country:
        return province + ', ' + country

data['Province/State'] = data.apply(combine_region, axis=1)

data.rename(columns={"Province/State": "Area"}, inplace=True)

data.replace('',0, inplace=True)

# Initiate variables
fig = go.Figure()
col_names = data.columns

# Looping for data splitting 
for i in range(4, len(data.columns)):
    df_split = data[['Lat','Long',
                   'Country/Region','Area',
                   col_names[i]]]
        
    df = df_split[(df_split[col_names[i]] != 0)].copy()
    lat = df['Lat']
    lon = df['Long']
    case = df[df.columns[-1]].astype(int)
    df['text'] = df['Area']+'<br>'+'confirmed cases: '+ case.astype(str)

    # Adding geoscatter plot
    fig.add_trace(go.Scattergeo(
        name = '',
        lon = lon,
        lat = lat,
        visible = False,
        hovertemplate = df['text'],
        text = df['text'],
        mode = 'markers',
        marker = {'size': 15,'opacity': 0.6,'color': 'Purple', 'symbol': 'circle'},
    ))

# Create slider
steps = []
for i in range(len(fig.data)):
    step = {
        "method": "restyle",
        "args": ["visible", [False] * len(fig.data)],
        "label": col_names[4:][i],
    }
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [{
    'active': 0,
    'currentvalue': {"prefix": "Date: "},
    'pad': {"t": 1},
    'steps': steps
}]

# Show data
fig.data[0].visible=True
fig.update_layout(sliders=sliders,title='Coronavirus Spreading Map'+'<br>geodose.com',height=600)
fig.show()
